In [31]:
# https://plot.ly/python/subplots/
import plotly as py
import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs
py.offline.init_notebook_mode(connected=True)

import scipy
import numpy as np
import pywt

In [7]:
class BaseWavelet:
    def __init__(self):
        self._lowpass = [np.sqrt(2) / 2] * 2 
        length = len(self._lowpass)
        self._highpass = [np.power(-1, i) * self._lowpass[length - i - 1] for i in range(length)]
        
    def forward(self, s, s_len):
        out = [0] * s_len
        
        lp_len = len(self._lowpass)
        n = len(s) // 2
        
        for i in range(n):
            start = i * 2
            for j in range(lp_len):
                k = start + j
                while k >= s_len:
                    k -= s_len
                
                out[i] += s[k] * self._lowpass[j]
                out[i + n] += s[k] * self._highpass[j]
        
        return out
    
class HaarWavelet(BaseWavelet):
    pass


In [8]:
class WaveletTransform:
    def __init__(self):
        self._wavelet = BaseWavelet()
    
    def forward(self, s, level = 1):
        s_len = len(s)
        out = np.zeros((level, s_len))
        s_copy = list(s)
        
        l = 0 #current level
        while s_len >= 2 and l < level:
            tmp = self._wavelet.forward(s_copy, s_len)
            tmp *= 2
            for i in range(s_len):
                s_copy[i] = tmp[i]
            out[l] = s_copy
            s_len // 2
            l += 1
        
        return out
        

In [9]:
N = 1024
T = 8#s
f = 1
w = 2*np.pi*f

t = np.linspace(0, T, N)

s = [np.sin(w * t[i]) if i < 512 else np.sin(4*w*t[i]) for i in range(N)]

wave = WaveletTransform().forward(s, 7)

trace = go.Heatmap(
    z=wave, 
    colorscale='Greys', 
    reversescale=True
)

signal = go.Scatter( 
    x = t,
    y = s,
    name='signal',
    mode='markers'
)

rows = 2
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)
fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(signal, 1, 1)
fig.append_trace(trace, 2, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [27]:
def mexican_hat(a, sigma = 1):
    return lambda t : 2/np.sqrt(3 * sigma)*np.power(np.pi, -0.25) * (1 - ((t/a) / sigma)**2)*np.exp(-(t/a)**2/(2*sigma**2))

def cwt(t, s, level = 1):
    out = np.zeros((level, len(s)))
    for i in range(level):
        wlt = mexican_hat(i + 1)
        wlt_data = [wlt(t_) for t_ in t]
        out[i] = 1/np.sqrt(i + 1) * np.convolve(s, wlt_data, mode='same')
        
    return out


In [29]:
N = 1024
T = 8#s
f = 1
w = 2*np.pi*f

t = np.linspace(0, T, N)

s = [np.sin(w * t[i]) if i < 512 else np.sin(4*w*t[i]) for i in range(N)]


wave = cwt(t, s, 8)

trace = go.Heatmap(
    z=wave, 
    colorscale='Greys', 
    reversescale=True
)

signal = go.Scatter( 
    x = t,
    y = s,
    name='signal',
    mode='markers'
)

wlt = mexican_hat(i + 1)
wlt_data = [wlt(t_) for t_ in t]
    
rows = 2
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)
fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(signal, 1, 1)
fig.append_trace(trace, 2, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [41]:
N = 999
T = 8#s
f = 1
w = 2*np.pi*f

t = np.linspace(0, T, N)

s = [np.sin(w * t[i]) if i < 333 or i > 666 else np.sin(4*w*t[i]) for i in range(N)]


widths = np.arange(1, 31)
cwtmatr, freqs = pywt.cwt(s, widths, 'mexh')

trace = go.Heatmap(
    z=cwtmatr, 
    colorscale='Greys', 
    reversescale=True
)

signal = go.Scatter( 
    x = t,
    y = s,
    name='signal',
    mode='markers'
)
    
rows = 2
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)
fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(signal, 1, 1)
fig.append_trace(trace, 2, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]

